# GlassDoor Data Download, Cleanup, and CSV Output

## Install Dependencies
Remove commented code to install dependencies, if needed

In [ ]:
# Install kagglehub if you need it, just comment out the next line
# !pip install kagglehub

## Download Data into Dataframe
Download the data from Kaggle and store into a dataframe 

In [ ]:
import kagglehub

# Download the Glassdoor Job Reviews dataset from Kaggle
path = kagglehub.dataset_download("davidgauthier/glassdoor-job-reviews-2")

print("Path to dataset files:", path)

In [ ]:
# Convert the csv at the variable "path" to a pd dataframe
import pandas as pd
import os

jobs_org_df = pd.read_csv(os.path.join(path, "all_reviews.csv"))

In [ ]:
# Display the first few rows of the cleaned dataset
jobs_org_df.head()

## Extract Company/Firm Names
Convert the links to the firm names into actual firm names and store into a new column

In [ ]:
# Function to extract and format the firm name
def extract_firm_name(link):
    # Split string on slashes
    parts = link.split("/")
    # Extract the last part of the string (the firm name)
    firm_name = parts[-1]
    # Split file name on dashes
    firm_name_parts = firm_name.split("-")
    # Remove the last 2 parts (the file extension) and join the rest with spaces
    firm_name = " ".join(firm_name_parts[:-2])
    # Return the formatted firm name
    return firm_name

# Test the function with different types of links    
print(f"Extract firm from relative path: {extract_firm_name('Reviews/Baja-Steel-and-Fence-Reviews-E5462645.htm')}")
print(f"Extract firm from absolute path: {extract_firm_name('https://www.glassdoor.com/Reviews/Calgary-Flames-Reviews-E5247.htm')}")


In [ ]:
# Transform the data set to extract the firm name from the link
jobs_org_df["firm_name"] = jobs_org_df["firm_link"].apply(extract_firm_name)

In [ ]:
def display_data_info(df):
    # Print Unique firm names
    print(f"\nUnique firm names: {df['firm_name'].unique()}")
    # Print the number of unique firm names
    print(f"\nNumber of unique firm names: {df['firm_name'].nunique()}")
    # Print the number of reviews per firm
    print(f"\nNumber of reviews per firm: {df['firm_name'].value_counts()}")
    # Get the count of total reviews
    print(f"\nTotal reviews: {df['firm_name'].count()}")

In [ ]:
display_data_info(jobs_org_df)

## Data Cleanup
CLean up unnecessary columns, drop nulls, and only take a minimum of 1,000 reviews per company

In [ ]:
# Drop any rows where the count of the firm name is less than 1,000
jobs_df = jobs_org_df[jobs_org_df["firm_name"].map(jobs_org_df["firm_name"].value_counts()) >= 1000]

In [ ]:
# Count the number of null or NaN entries for each column
print(f"\nCount of null or NaN entries for each column:\n{jobs_df.isnull().sum()}")

In [ ]:
# Let's drop the columns that have almost all NULL values
jobs_df = jobs_df.drop(columns=["advice", "index"])

In [ ]:
# Drop any rows with null or NaN entries in any column
jobs_df = jobs_df.dropna()

# Count the number of null or NaN entries for each column
print(f"\nCount of null or NaN entries for each column:\n{jobs_df.isnull().sum()}")

In [ ]:
# Get counts of each unique values in the "status" column
print(f"\nCounts of each unique value in the 'status' column:\n{jobs_df['status'].value_counts()}")


In [ ]:
# Drop any rows where the count of the firm name is less than 1,000 after cleaning
jobs_df = jobs_df[jobs_df["firm_name"].map(jobs_df["firm_name"].value_counts()) >= 1000]

In [ ]:
print("AFTER DATA CLEANING:")
display_data_info(jobs_df)

## Data Cleanup - Bias Control
Only take 1,000 reviews per company to try and reduce bias for large companies

In [ ]:
# limit the dataset to 1,000 random entries per company listed in the firm_name column
jobs_max1000_df = jobs_df.groupby("firm_name").apply(lambda x: x.sample(n=1000, random_state=42)).reset_index(drop=True)


In [ ]:
print("AFTER REDUCING EACH REVIEW COUNT TO 1,000 FOR EACH COMPANY:")
display_data_info(jobs_max1000_df)

In [ ]:
# Create a "Resources" folder in the local directory if it does not exist
if not os.path.exists("Resources"):
    os.makedirs("Resources")

# Export the cleaned dataset to a CSV file
jobs_max1000_df.to_csv("Resources/cleaned_glassdoor_reviews_max1000.csv", index=False)

## Further Reduce the Data to Help with Model Consumption
Further reduce the dataset because Google Collab was having problems with the larger file

### Chunking approach
Save dataset into several files with a max of 250,000 reviews

In [ ]:
# Get the size of the jobs_df dataframe
total_rows = jobs_max1000_df.shape[0]
rows_step = 25000
start_index = 0
step_number = 0

# Create a "Resources" folder in the local directory if it does not exist
if not os.path.exists("Resources"):
    os.makedirs("Resources")

# Create a "chunked" folder in the Resources directory if it does not exist
if not os.path.exists("Resources/chunked"):
    os.makedirs("Resources/chunked")

# Loop through the dataframe in chunks of 250,000 rows
while start_index < total_rows:
    end_index = min(start_index + rows_step, total_rows)
    chunk_df = jobs_max1000_df.iloc[start_index:end_index]
    
    # Save the chunk to a CSV file
    chunk_df.to_csv(f"Resources/chunked/cleaned_glassdoor_reviews_max1000_{start_index}.csv", index=False)
    
    # Print Progress
    print(f"Saved chunk {step_number} from index {start_index} to {end_index}")
    print(f"Chunk {step_number} shape: {chunk_df.shape}")
    
    # Update the start index for the next chunk
    start_index += rows_step

    # Increase the step number for the next file name
    step_number += 1
    

### Sample Approach
Reduce to a random sample of 500,000 reviews and only take companies with a minimum of 500 Reviews, and reduce all review counts to 500

In [ ]:
# Get a sample of 500,000 reviews
jobs_sample_df = jobs_df.sample(n=500000, random_state=1)

In [ ]:
# Drop any rows where the count of the firm name is less than 500
jobs_sample_reduced_df = jobs_sample_df[jobs_sample_df["firm_name"].map(jobs_sample_df["firm_name"].value_counts()) >= 500]

In [ ]:
# limit the dataset to 500 random entries per company listed in the firm_name column
jobs_sample_reduced_df = jobs_sample_reduced_df.groupby("firm_name").apply(lambda x: x.sample(n=500, random_state=42)).reset_index(drop=True)

In [ ]:
print("AFTER DATA REDUCTION AND A LIMIT OF 500 REVIEWS:")
display_data_info(jobs_sample_reduced_df)

In [ ]:
# Create a "Resources" folder in the local directory if it does not exist
if not os.path.exists("Resources"):
    os.makedirs("Resources")
    
# Export the cleaned dataset to a CSV file
jobs_sample_reduced_df.to_csv("Resources/cleaned_glassdoor_reviews_reduced_max500.csv", index=False)